In [28]:
## AUTHOR: Àlex López Diaz
## Date: 20/11/2022

In [18]:
# Imports necessaris
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from random import randint
from time import sleep

# URLs a UTILITZAR
URL_SCRAP = 'https://marvel.fandom.com/wiki/Character_Index'
URL_SCRAP_A = 'https://marvel.fandom.com/wiki/Character_Index/B'
CSV_PATH = './Result/'
CSV_NAME = 'result_web_scrap_marvel.csv'

In [19]:
#######################################################
##                    CAS BASE                       ##
#######################################################

In [20]:
# CAS BASE
# Utilitzarem el cas base d'extraure l'informació d'una pàgina en contret
# per automatitzar-ho més endavant

page = requests.get(URL_SCRAP_A)
soup = BeautifulSoup(page.content)
#print(soup.prettify())

In [21]:
# Extraiem els fragments que contenen la informació dels personatges
characters_soup = soup.find_all("span",{"class": "mw-headline"})
#print(characters_soup)

In [22]:
# Funció que  extreu els noms dels personatges
# Al estar la info del personatge i el nom en grups separats de l'html,
# farem dos datasets amb la mateixa llargada i farem un "merge"
def extract_characters_names(soup):
    characters_soup = soup.find_all("span",{"class": "mw-headline"})
    charecters_names = []
    for character in characters_soup:
        if not character.string.startswith("Character Index",0,15) and character.string != 'References':
            charecters_names.append(character.string)
    return charecters_names

characters_names = extract_characters_names(soup)
#print(characters_names)

In [23]:
# Constructor de cada linia
# Passem el nom per paràmetre per afegir-lo al registre
def extract_character_info(name,tr):
    # Hi ha registres sense foto
    if tr.find("a", {"class": "image"}) is None:
        image = ''
    else:
        image = tr.find("a", {"class": "image"})["href"]
    # Hi ha registres sense appearences
    if tr.find("a", title = True) is None:
        appearence = ''
    else:
        appearence = tr.find("a", title = True)["title"]
    # Hi ha registres sense descripcio
    if tr.find("p") is None:
        description = ''
    else:
        description = tr.find("p").text.strip()
        
    character_data = {
        'name': name,
        'image': image,
        'appearences': appearence,
        'description': description,
    }
    return character_data

# Constructor de les linies de cada bloc
# Itera tots els blocs d'informació per crear un registre per cada un
def exctract_all_character_info(name, soup):
    character_data = []
    for i in soup.select("tr"):
        character_data.append(extract_character_info(name, i))
    
    return character_data

In [26]:
# Prova per comprovar que les funcions funcionen
character_data_soup = soup.find_all("table",{"class": "article-table"})
#print(exctract_all_character_info(characters_names[0],character_data_soup[3]))

In [27]:
# Confirmem que els dos datasets, data i names tenen la mateixa llargada
#print(len(character_data_soup))
#print(len(characters_names))

In [28]:
# Funció que extreu tota la informació de la pàgina web
# Crida a la funció exctract_all_character_info que crea registres per 
# cada article. Necessita un nom per afegir-lo.
def scrap_web_with_info(soup):
    character_data_soup = soup.find_all("table",{"class": "article-table"})
    characters_names = extract_characters_names(soup)
    result = []
    for i in range(0,len(character_data_soup)-1):
        result = result + exctract_all_character_info(characters_names[i], character_data_soup[i])
    return result

#print(len(scrap_web_with_info(soup)))

# Cridant aquesta funció podem realitzar l'extracció del cas base.
result = scrap_web_with_info(soup)


In [29]:
# Funció per exportar el dataset a un csv
def export_data(result):
    header = result[0].keys()
    dir_path = os.getcwd()
    df = pd.DataFrame(result)
    df.to_csv(CSV_PATH + CSV_NAME, sep=";", header=header)

In [30]:
#######################################################
##             PROCEDIMENT PRINCIPAL                 ##
#######################################################

In [31]:
# Obtenim les dades de la web índex
page = requests.get(URL_SCRAP)
soup = BeautifulSoup(page.content)
#print(soup.prettify())

In [32]:
# Busquem el nom dels index a afegir a la url base
# Cada web que abans hem anomenat "cas base" té per URLla combinació
# de la URL base + l'identificador de l'índex
sufixes_to_add = []

for i in soup.find_all("span",{"class": "mw-headline"}):
    if i.select("a"):
        sufixes_to_add.append(i["id"])

In [33]:
#print(sufixes_to_add)

In [34]:
#SCRAP!!!!!!!

result_final = []
# Capem per no iterar totes les pàgines
for i in range(0,3): # Canviar el 3 per 26 si es vol l'execució completa
    # Afegim el valor recollit a la url base
    url = URL_SCRAP + '/' + sufixes_to_add[i]
    print("Scrapping page: " + URL_SCRAP + '/' + sufixes_to_add[i])
    
    i_page = requests.get(url)
    i_soup = BeautifulSoup(i_page.content)
    
    # Extraiem la informació cridant a la funció scrap_web_with_info
    result_final = result_final + scrap_web_with_info(i_soup)
    
    # Realitzem parades random entre 1 i 5 segons per evitar bloquejos
    aux = randint(1,5)
    print(f"Sleep for: {aux} seconds.")
    sleep(aux)
    
export_data(result_final)
print("TOT OK")

Scrapping page: https://marvel.fandom.com/wiki/Character_Index/A
Sleep for: 2 seconds.
Scrapping page: https://marvel.fandom.com/wiki/Character_Index/B
Sleep for: 2 seconds.
Scrapping page: https://marvel.fandom.com/wiki/Character_Index/C
Sleep for: 3 seconds.
TOT OK
